In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score
import pickle
from collections import defaultdict
from sklearn.utils import shuffle
import os
import cv2
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd

from torchinfo import summary
from sklearn.metrics import confusion_matrix
import seaborn as sns



In [2]:
import numpy as np
import os

# Define the folder path
folder_path = r"D:\PYTHONIG\newwindow\NOTEBOOKS_2025\aprilmay2025\data\numpy\optunadata\mel"

# Load the numpy files into the respective arrays with the correct capitalized naming
eeg_fold_1 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_1.npy'))
labels_fold_1 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_1.npy'))
patients_fold_1 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_1.npy'))

eeg_fold_2 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_2.npy'))
labels_fold_2 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_2.npy'))
patients_fold_2 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_2.npy'))

eeg_fold_3 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_3.npy'))
labels_fold_3 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_3.npy'))
patients_fold_3 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_3.npy'))

eeg_fold_4 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_4.npy'))
labels_fold_4 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_4.npy'))
patients_fold_4 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_4.npy'))

eeg_fold_5 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_5.npy'))
labels_fold_5 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_5.npy'))
patients_fold_5 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_5.npy'))

eeg_folds = [eeg_fold_1, eeg_fold_2, eeg_fold_3, eeg_fold_4, eeg_fold_5]
labels_folds = [labels_fold_1, labels_fold_2, labels_fold_3, labels_fold_4, labels_fold_5]
patients_folds = [patients_fold_1, patients_fold_2, patients_fold_3, patients_fold_4, patients_fold_5]

for i in range(len(eeg_folds)):
    eeg_folds[i] = eeg_folds[i].astype(np.float16)

In [3]:
def data_balancer(data, labels, factor):
    # Count the number of samples in each class
    num_class_0 = np.sum(labels == 0)
    num_class_1 = np.sum(labels == 1)
    num_class_2 = np.sum(labels == 2)

    # Find the minimum number of samples across all classes
    min_samples = min(num_class_0, num_class_1, num_class_2)

    # Calculate the number of samples to take from each class
    samples_per_class = min_samples // factor

    # Randomly sample 'samples_per_class' from each class
    class_0_indices = np.random.choice(np.where(labels == 0)[0], samples_per_class, replace=False)
    class_1_indices = np.random.choice(np.where(labels == 1)[0], samples_per_class, replace=False)
    class_2_indices = np.random.choice(np.where(labels == 2)[0], samples_per_class, replace=False)

    # Combine balanced indices
    balanced_indices = np.concatenate((class_0_indices, class_1_indices, class_2_indices))

    # Shuffle the balanced indices
    np.random.shuffle(balanced_indices)

    # Create balanced training data and labels
    balanced_data = data[balanced_indices]
    balanced_labels = labels[balanced_indices]

    return balanced_data, balanced_labels

In [4]:
class EarlyStopping:
    def __init__(self, patience=5):
        """
        Initializes the early stopping mechanism based on divergence detection.

        Args:
            patience (int): Number of consecutive epochs with increasing validation loss
                            before stopping.
        """
        self.patience = patience
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, model):
        """
        Checks if the validation loss is diverging and updates the state accordingly.

        Args:
            val_loss (float): Current epoch's validation loss.
            model (torch.nn.Module): The model being trained.
        """
        if self.best_loss is None or val_loss < self.best_loss:
            # Improvement detected
            self.best_loss = val_loss
            self.best_model_state = model.state_dict()
            self.counter = 0
        else:
            # Validation loss increased
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Divergence detected. Stopping training after {self.counter} epochs.")
                self.early_stop = True

    def load_best_model(self, model):
        """
        Restores the model to the state with the lowest validation loss.

        Args:
            model (torch.nn.Module): The model to restore.
        """
        model.load_state_dict(self.best_model_state)




In [5]:
import torch
import torch.nn as nn

debug_mode_flag = False
# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#set random seed for reproducibility
torch.manual_seed(42)
debug_mode_flag = False


def patchify(data, n_patches):
    n, c, h, w = data.shape
    # Create tensor on the same device as input data
    patches = torch.zeros(n, n_patches**2, (c*h*w) // (n_patches**2), device=data.device)
    patch_size = h // n_patches
    for idx, d in enumerate(data):
        for i in range(n_patches):
            for j in range(n_patches):
                patch = d[:, i*patch_size: (i+1)*patch_size, j*patch_size: (j+1) * patch_size]
                patches[idx, i*n_patches + j] = patch.flatten()
    return patches

def get_positional_embeddings(sequence_length, d, device='cpu'):
    result = torch.ones(sequence_length, d, device=device)
    for i in range(sequence_length):
        for j in range(d):
            result[i][j] = np.sin(i / (10000 ** (j / d))) if j % 2 == 0 else np.cos(i / (10000 ** ((j - 1) / d)))
    return result
class vit(nn.Module):
    def __init__(self, input_size, n_patches, hidden_size, num_classes, num_transformer_blocks=2, num_heads=4):
        super().__init__()
        self.chw = input_size
        self.n_patches = n_patches
        self.patch_size = ((self.chw[1] // n_patches) ** 2) * self.chw[0]
        
        # Core components
        self.linear_mapper = nn.Linear(self.patch_size, hidden_size)
        self.class_token = nn.Parameter(torch.rand(1, hidden_size))
        
        # Simplified positional embedding
        self.register_buffer('pos_embed', get_positional_embeddings(self.n_patches ** 2 + 1, hidden_size))
        
        # Normalization and Transformer
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_size, nhead=num_heads, batch_first=True),
            num_layers=num_transformer_blocks
        )
        
        # Classification head
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Create patches
        patches = patchify(x, self.n_patches)
        
        # Project patches to embedding dimension
        x = self.linear_mapper(patches)
        
        # Add class token
        batch_size = x.shape[0]
        cls_tokens = self.class_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        
        # Add positional embeddings
        x = x + self.pos_embed
        x = self.layer_norm(x)
        
        # Apply transformer
        x = self.transformer(x)
        
        # Classification based on CLS token
        x = x[:, 0]
        return self.classifier(x)

In [6]:
from torchinfo import summary
model = vit(input_size=(3, 224, 224), n_patches=14, hidden_size=256, num_classes=3,num_transformer_blocks=12)
summary(model)

Layer (type:depth-idx)                                            Param #
vit                                                               256
├─Linear: 1-1                                                     196,864
├─LayerNorm: 1-2                                                  512
├─TransformerEncoder: 1-3                                         --
│    └─ModuleList: 2-1                                            --
│    │    └─TransformerEncoderLayer: 3-1                          1,315,072
│    │    └─TransformerEncoderLayer: 3-2                          1,315,072
│    │    └─TransformerEncoderLayer: 3-3                          1,315,072
│    │    └─TransformerEncoderLayer: 3-4                          1,315,072
│    │    └─TransformerEncoderLayer: 3-5                          1,315,072
│    │    └─TransformerEncoderLayer: 3-6                          1,315,072
│    │    └─TransformerEncoderLayer: 3-7                          1,315,072
│    │    └─TransformerEncoderLayer: 3-8  

In [7]:
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# pca = sklearn.decomposition.PCA(3)
epochs = 100


fold_indices = np.arange(5)
fold_indices = np.random.permutation(fold_indices)
val_fold_indices = np.roll(fold_indices, 1)

input_shape = (3,224,224)
num_classes = 3


In [8]:
def objective(trial):
    # Focused hyperparameter space
    n_patches = trial.suggest_categorical("n_patches", [14, 28])
    embed_dim = trial.suggest_categorical("embed_dim", [64, 128, 256])
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-4, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32])
    num_transformer_blocks = trial.suggest_int("num_transformer_blocks", 2, 8, step=2)
    num_heads = trial.suggest_categorical("num_heads", [4, 8])
    
    # Fixed parameters to reduce search space
    optimizer_name = "Adam"
    label_smoothing = 0.1
    factor = 1
    
    print(f"Trial {trial.number} params: n_patches={n_patches}, embed_dim={embed_dim}, lr={learning_rate}, "
          f"blocks={num_transformer_blocks}, heads={num_heads}, batch_size={batch_size}")
    
    fold_accuracies = []

    for test_fold_idx in range(5):
        # Get train/val/test folds (keeping existing logic)
        test_fold = fold_indices[test_fold_idx]
        remaining_folds = [fold_indices[i] for i in range(5) if i != test_fold_idx]
        val_fold = remaining_folds[test_fold_idx % 4]
        train_folds = [fold for fold in remaining_folds if fold != val_fold]
        
        # Prepare data
        train_data = np.concatenate([eeg_folds[j] for j in train_folds]).transpose(0, 3, 1, 2)
        train_labels = np.concatenate([labels_folds[j] for j in train_folds])
        val_data = eeg_folds[val_fold].transpose(0, 3, 1, 2)
        val_labels = labels_folds[val_fold]
        test_data = eeg_folds[test_fold].transpose(0, 3, 1, 2)
        test_labels = labels_folds[test_fold]
        
        # Balance training data
        balanced_train_data, balanced_train_labels = data_balancer(train_data, train_labels, factor)
        
        # Create data loaders
        train_loader = DataLoader(
            TensorDataset(torch.tensor(balanced_train_data, dtype=torch.float32),
                          torch.tensor(balanced_train_labels, dtype=torch.long)),
            batch_size=batch_size, shuffle=True, pin_memory=True
        )
        
        val_loader = DataLoader(
            TensorDataset(torch.tensor(val_data, dtype=torch.float32),
                          torch.tensor(val_labels, dtype=torch.long)),
            batch_size=batch_size, shuffle=False, pin_memory=True
        )
        
        test_loader = DataLoader(
            TensorDataset(torch.tensor(test_data, dtype=torch.float32),
                          torch.tensor(test_labels, dtype=torch.long)),
            batch_size=batch_size, shuffle=False, pin_memory=True
        )
        
        # Initialize model
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = vit(input_size=input_shape, n_patches=n_patches, hidden_size=embed_dim,
                    num_classes=num_classes, num_transformer_blocks=num_transformer_blocks,
                    num_heads=num_heads).to(device)
        
        # Set up training
        criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        early_stopping = EarlyStopping(patience=5)  # Reduced patience
        
        # Training loop
        for epoch in range(25):  # Reduced max epochs
            # Train
            model.train()
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            
            # Validate
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for val_inputs, val_labels in val_loader:
                    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                    val_outputs = model(val_inputs)
                    val_loss += criterion(val_outputs, val_labels).item()
            
            val_loss /= len(val_loader)
            early_stopping(val_loss, model)
            
            if early_stopping.early_stop:
                break
        
        # Load best model and evaluate
        early_stopping.load_best_model(model)
        model.eval()
        all_preds, all_labels = [], []
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        fold_acc = balanced_accuracy_score(all_labels, all_preds)
        fold_accuracies.append(fold_acc)
        print(f"Fold {test_fold_idx+1}: Accuracy = {fold_acc:.4f}")
        
        # Clean up
        del model
        torch.cuda.empty_cache()
    
    mean_accuracy = np.mean(fold_accuracies)
    print(f"Trial {trial.number} mean accuracy: {mean_accuracy:.4f}")
    
    return mean_accuracy

In [9]:
import optuna
# Streamlined Optuna configuration
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=2)
)

# Reduced number of trials for faster convergence
study.optimize(objective, n_trials=100, n_jobs=2)

print("Best hyperparameters:", study.best_params)
print(f"Best accuracy: {study.best_value:.4f}")

c:\Users\Gabriel\anaconda3\envs\cudaenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-05 02:23:49,748] A new study created in memory with name: no-name-c636610a-b1f0-488e-9865-d46557ffa0fc


Trial 0 params: n_patches=14, embed_dim=256, lr=0.0001113753290591893, blocks=2, heads=8, batch_size=32Trial 1 params: n_patches=28, embed_dim=64, lr=2.8167148948507208e-05, blocks=4, heads=4, batch_size=32

Divergence detected. Stopping training after 5 epochs.
Fold 1: Accuracy = 0.4255
Divergence detected. Stopping training after 5 epochs.
Fold 2: Accuracy = 0.3531
Divergence detected. Stopping training after 5 epochs.
Fold 1: Accuracy = 0.3842
Divergence detected. Stopping training after 5 epochs.
Fold 3: Accuracy = 0.3462
Divergence detected. Stopping training after 5 epochs.
Fold 4: Accuracy = 0.5272
Divergence detected. Stopping training after 5 epochs.
Fold 2: Accuracy = 0.3444
Divergence detected. Stopping training after 5 epochs.


[I 2025-05-05 02:30:16,380] Trial 0 finished with value: 0.4326567967259911 and parameters: {'n_patches': 14, 'embed_dim': 256, 'learning_rate': 0.0001113753290591893, 'weight_decay': 2.0818818082475494e-05, 'batch_size': 32, 'num_transformer_blocks': 2, 'num_heads': 8}. Best is trial 0 with value: 0.4326567967259911.


Fold 5: Accuracy = 0.5113
Trial 0 mean accuracy: 0.4327
Trial 2 params: n_patches=28, embed_dim=256, lr=0.00019769906034741944, blocks=4, heads=4, batch_size=16
Divergence detected. Stopping training after 5 epochs.
Fold 3: Accuracy = 0.3383
Divergence detected. Stopping training after 5 epochs.
Fold 1: Accuracy = 0.3657
Divergence detected. Stopping training after 5 epochs.
Fold 4: Accuracy = 0.4807
Divergence detected. Stopping training after 5 epochs.
Fold 2: Accuracy = 0.3362
Divergence detected. Stopping training after 5 epochs.
Fold 3: Accuracy = 0.3935
Divergence detected. Stopping training after 5 epochs.
Fold 4: Accuracy = 0.5044
Divergence detected. Stopping training after 5 epochs.


[I 2025-05-05 02:48:11,987] Trial 1 finished with value: 0.3971686061386811 and parameters: {'n_patches': 28, 'embed_dim': 64, 'learning_rate': 2.8167148948507208e-05, 'weight_decay': 3.0096891700188963e-05, 'batch_size': 32, 'num_transformer_blocks': 4, 'num_heads': 4}. Best is trial 0 with value: 0.4326567967259911.


...

Fold 5: Accuracy = 0.3848
Trial 98 mean accuracy: 0.3755
Trial 99 params: n_patches=14, embed_dim=64, lr=1.2342942482714458e-05, blocks=4, heads=8, batch_size=32
Divergence detected. Stopping training after 5 epochs.
Fold 1: Accuracy = 0.3038
Divergence detected. Stopping training after 5 epochs.


[I 2025-05-05 12:00:06,601] Trial 97 finished with value: 0.3813560293846473 and parameters: {'n_patches': 14, 'embed_dim': 64, 'learning_rate': 1.3401405071919073e-05, 'weight_decay': 5.700083076772603e-05, 'batch_size': 16, 'num_transformer_blocks': 6, 'num_heads': 8}. Best is trial 74 with value: 0.4339657674596612.


Fold 5: Accuracy = 0.3925
Trial 97 mean accuracy: 0.3814
Divergence detected. Stopping training after 5 epochs.
Fold 2: Accuracy = 0.4459
Divergence detected. Stopping training after 5 epochs.
Fold 3: Accuracy = 0.3605
Divergence detected. Stopping training after 5 epochs.
Fold 4: Accuracy = 0.5069
Divergence detected. Stopping training after 5 epochs.


[I 2025-05-05 12:02:33,777] Trial 99 finished with value: 0.4139751453665611 and parameters: {'n_patches': 14, 'embed_dim': 64, 'learning_rate': 1.2342942482714458e-05, 'weight_decay': 2.3932751754652317e-05, 'batch_size': 32, 'num_transformer_blocks': 4, 'num_heads': 8}. Best is trial 74 with value: 0.4339657674596612.


Fold 5: Accuracy = 0.4528
Trial 99 mean accuracy: 0.4140
Best hyperparameters: {'n_patches': 14, 'embed_dim': 64, 'learning_rate': 4.791527211534685e-05, 'weight_decay': 2.723389186594195e-05, 'batch_size': 32, 'num_transformer_blocks': 6, 'num_heads': 8}
Best accuracy: 0.4340
